In [1]:
#SETUP
from google.cloud import bigquery
from google.cloud.bigquery import Row
from google.cloud import storage
from datetime import time, datetime
import sys
import csv

In [2]:
#NOMBRES VARIABLES PARA TABLA TEMPORAL
par_db = "4040"
par_periodo_d = datetime.now().strftime("%Y-%m-%d")
par_periodo_t = datetime.now().strftime("%Y_%m_%d")

In [3]:
#VARIABLES
project_id = "dfa-dna-ws0009-la-prd-d0bb"
dataset_id = "dfa_dna_ws0009_la_prd_sandbox"
dataset_landing_id = "dfa-dna-ws0009-la-prd-landing-zone"
location = "northamerica-northeast1"

#NOMBRES CSV DE SALIDA
input_filename = "t1_in.csv"
tmp_filename = "ignite_portfolio_bcu_tmp.csv"
output_filename = f"ignite_portfolio_bcu_out_{par_db}_{par_periodo_t}"

#NOMBRE DE CARPETAS
input_folder = f"ignite_portfolio_bcu/in/"
tmp_folder = f"ignite_portfolio_bcu/tmp/"
out_folder = f"ignite_portfolio_bcu/out/"

#TABLES NAMES
table_name_query_1 =  "uy_efx_bcu_hash_la_prd"
table_name_query_2 =  "uy_efx_credito_weekly_la_prd"
table_name_temp_sandbook = f"tmp_input_cliente_ignite_portfolio_bcu_{par_db}_{par_periodo_t}"
table_name_temp_sandbook_s = f"tmp_input_cliente_ignite_portfolio_bcu_soporte"

In [6]:
#CREAR TABLA SOPRTE
schema = [
    bigquery.SchemaField("ID", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("documento", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("MARCA_FINAL", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("TIPO_FICHA", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("ARCHIVE", "STRING", mode="NULLABLE"),
]

#TABLA NAME
client = bigquery.Client()
tmp_table_name = project_id + "." + dataset_id + "." + table_name_temp_sandbook_s
table = bigquery.Table(tmp_table_name, schema=schema)
table = client.create_table(table)

#TRAER UN BUCKET
storage_client = storage.Client()
gcs_bucket = dataset_landing_id

#CREAR SCHEMA
bucket = storage_client.get_bucket(gcs_bucket)
blob = bucket.get_blob(input_folder + input_filename)
downloaded_blob = blob.download_as_string()

#CARGAR CSV AL SCHEMA
gcs_bucket = dataset_landing_id
job_config = bigquery.LoadJobConfig(schema = schema, skip_leading_rows = 1, source_format = bigquery.SourceFormat.CSV, field_delimiter=",")
uri = f"gs://{gcs_bucket}/{input_folder + input_filename}"
load_job = client.load_table_from_uri(uri, tmp_table_name, job_config = job_config)  
load_job.result()  

#VERIFICACION
destination_table = client.get_table(tmp_table_name)  
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1598 rows.


In [8]:
#LLAMAR TABLAS
client = bigquery.Client()
table_id_1 = project_id + "." + dataset_id + "." + table_name_query_1
table_1 = client.get_table(table_id_1)

client = bigquery.Client()
table_id_2 = project_id + "." + dataset_id + "." + table_name_query_2
table_2 = client.get_table(table_id_2)

client = bigquery.Client()
table_id_3 = project_id + "." + dataset_id + "." + table_name_temp_sandbook_s
table_3 = client.get_table(table_id_3)

#QUERY IGNITE BCU
sql = f"SELECT  T1.documento \
        , T1.documento AS RUT \
        , T4.NOMBRE AS RAZON_SOCIAL \
        , T2.entidad \
        , T2.calificacion as calificacion_entidad \
        , TRUNC(t2.SALDO_VIGENTE_MN) AS SALDO_VIGENTE_MN \
        , TRUNC(t2.SALDO_VENCIDO_MN) AS SALDO_VENCIDO_MN \
        , TRUNC((t2.SALDO_VIGENTE_MN+t2.SALDO_VENCIDO_MN)) AS SALDO_MN \
        , TRUNC(t2.SALDO_VIGENTE_ME) AS SALDO_VIGENTE_ME \
        , TRUNC(t2.SALDO_VENCIDO_ME) AS SALDO_VENCIDO_ME \
        , TRUNC((t2.SALDO_VIGENTE_ME+t2.SALDO_VENCIDO_ME)) AS SALDO_ME \
        , T3.PEOR_BCU_TOTAL \
        , 1 as ACTIVIDAD \
        , 0 AS FALLECIDO \
        , T1.TIPO_FICHA \
FROM \
   `" + table_id_3 + "` T1 JOIN \
    ( \
    SELECT  T1.PERS_ID \
            , UPPER(T2.ENTIDAD.NOMBRE) AS ENTIDAD \
            , T2.CALIFICACION \
            , SUM(CASE WHEN T3.CODIGO = 12410009  AND T3.MONEDA = 'MN' THEN T3.IMPORTE ELSE 0 END) AS SALDO_VIGENTE_MN \
            , SUM(CASE WHEN T3.CODIGO = 12600006  AND T3.MONEDA = 'MN' THEN T3.IMPORTE ELSE 0 END) AS SALDO_VENCIDO_MN \
            \
            , SUM(CASE WHEN T3.CODIGO = 12410009  AND T3.MONEDA = 'ME' THEN T3.IMPORTE ELSE 0 END) AS SALDO_VIGENTE_ME \
            , SUM(CASE WHEN T3.CODIGO = 12600006  AND T3.MONEDA = 'ME' THEN T3.IMPORTE ELSE 0 END) AS SALDO_VENCIDO_ME \
    FROM `" + table_id_1 + "` T1, \
            T1.DEUDAS T2, \
            T2.RUBROS T3 \
    WHERE T1.ARCHIVE = '2022-01-01' \
            AND T3.IMPORTE IS NOT NULL \
    group by T1.PERS_ID, t2.calificacion, T2.ENTIDAD.NOMBRE \
    ) T2 ON T1.documento = T2.PERS_ID  JOIN \
    ( \
    SELECT  T1.PERS_ID \
            , MAX(T2.CALIFICACION) AS PEOR_BCU_TOTAL \
    FROM `" + table_id_1 + "` T1, \
            T1.DEUDAS T2 \
    WHERE  T1.ARCHIVE = '2022-01-01' \
    group by T1.PERS_ID \
    ) T3 ON T1.documento = T3.PERS_ID \
    LEFT JOIN \
    ( \
        SELECT  PERS_ID \
            , TIPO_DOCUMENTO \
            , CASE WHEN T1.TIPO_DOCUMENTO = 'CI' THEN \
                    CONCAT \
                    ( ifnull(T1.APELLIDO_1ERO, '') \
                    , ifnull(T1.APELLIDO_2DO, '') \
                    , ifnull(T1.NOMBRE_1ERO, '') \
                    , ifnull(T1.NOMBRE_2DO, '') \
                    ) \
                    ELSE \
                    ifnull(T1.razon_social, '') \
                    END AS NOMBRE \
                    \
        FROM `" + table_id_2 + "` T1 \
    ) T4 ON T1.documento = T4.PERS_ID"

#CREAR DATAFRAME
df = client.query(sql).to_dataframe()

#SETUP UPLOAD
client = storage.Client()
bucket = client.get_bucket(dataset_landing_id)

#UPLOAD CSV TO BUCKET TEMP
bucket.blob(tmp_folder + tmp_filename).upload_from_string(df.to_csv(index = False, sep = ','), 'text/csv')

In [9]:
#CREAR TABLA TEMP SANDBOOK
schema = [
    bigquery.SchemaField("documento", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("RUT", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("RAZON_SOCIAL", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("entidad", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("calificacion_entidad", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_VIGENTE_MN", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_VENCIDO_MN", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_MN", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_VIGENTE_ME", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_VENCIDO_ME", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("SALDO_ME", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("PEOR_BCU_TOTAL", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("ACTIVIDAD", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("FALLECIDO", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("TIPO_FICHA", "STRING", mode="NULLABLE")
]

#TABLA NAME
client = bigquery.Client()
tmp_table_name = project_id + "." + dataset_id + "." + table_name_temp_sandbook
table = bigquery.Table(tmp_table_name, schema=schema)
table = client.create_table(table)

In [10]:
#TRAER UN BUCKET
storage_client = storage.Client()
gcs_bucket = dataset_landing_id

#CREAR SCHEMA
bucket = storage_client.get_bucket(gcs_bucket)
blob = bucket.get_blob(tmp_folder + tmp_filename)
downloaded_blob = blob.download_as_string()

#CARGAR CSV AL SCHEMA
gcs_bucket = dataset_landing_id
job_config = bigquery.LoadJobConfig(schema = schema, skip_leading_rows = 1, source_format = bigquery.SourceFormat.CSV)
uri = f"gs://{gcs_bucket}/{tmp_folder + tmp_filename}"
load_job = client.load_table_from_uri(uri, tmp_table_name, job_config = job_config)  
load_job.result()  

#REVISAR
destination_table = client.get_table(tmp_table_name)  
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 0 rows.


In [12]:
#GET TABLE
client = bigquery.Client()
table_id_1 = project_id + "." + dataset_id + "." + table_name_temp_sandbook
table_1 = client.get_table(table_id_1)

sql = f"SELECT * FROM {table_id_1}"

#CREAR DATAFRAME
df = client.query(sql).to_dataframe()

#SETUP UPLOAD
client = storage.Client()
bucket = client.get_bucket(dataset_landing_id)

#UPLOAD CSV TO BUCKET TEMP
bucket.blob(out_folder + output_filename).upload_from_string(df.to_csv(index = False, sep = ','), 'text/csv')

In [14]:
#DELETE IN CSV
gcs_bucket = dataset_landing_id
bucket1 = storage_client.get_bucket(gcs_bucket)
blob1 = bucket1.get_blob(input_folder + input_filename)
blob1.delete()

#DELETE TEMP CSV
buckett = storage_client.get_bucket(gcs_bucket)
blobt = buckett.get_blob(tmp_folder + tmp_filename)
blobt.delete()

print("Deleted csv '{}'.".format(blob1))
print("Deleted csv '{}'.".format(blobt))

Deleted csv '<Blob: dfa-dna-ws0009-la-prd-landing-zone, ignite_portfolio_bcu/in/t1_in.csv, 1658436847779280>'.
Deleted csv '<Blob: dfa-dna-ws0009-la-prd-landing-zone, ignite_portfolio_bcu/tmp/ignite_portfolio_bcu_tmp.csv, 1658437019240865>'.


In [15]:
#BORRAR TABLAS EN SANDBOOK
#APLICAR DESPUES DE OCUPAR LA TABLA
client = bigquery.Client()

table_id = (project_id + "." + dataset_id + "." + table_name_temp_sandbook)
client.delete_table(table_id, not_found_ok = True)

table_id_s = (project_id + "." + dataset_id + "." + table_name_temp_sandbook_s)
client.delete_table(table_id_s, not_found_ok = True)

print("Deleted table '{}'.".format(table_id))
print("Deleted table '{}'.".format(table_id_s))

print("Proceso finalizado con exito, revisar csv carpeta ignite_portfolio_bcu/out")

Deleted table 'dfa-dna-ws0009-la-prd-d0bb.dfa_dna_ws0009_la_prd_sandbox.tmp_input_cliente_ignite_portfolio_bcu_4040_2022_07_21'.
Deleted table 'dfa-dna-ws0009-la-prd-d0bb.dfa_dna_ws0009_la_prd_sandbox.tmp_input_cliente_ignite_portfolio_bcu_soporte'.
Proceso finalizado con exito, revisar csv carpeta ignite_portfolio_bcu/out
